Autor: Matyáš Sládek <br>
Rok: 2020 <br>

Tento soubor slouží k predicki žánrů skladeb lokálního archivu a jejich anotaci.

Tato buňka importuje potřebné knihovny.

In [1]:
import sys
import json
import joblib

import numpy as np
import pandas as pd

from mutagen.easyid3 import EasyID3

from sklearn.preprocessing import LabelEncoder, StandardScaler

Tato buňka obsahuje funkci pro predicki žánrů skladeb z lokálního archivu, umožňuje také modifikovat ID3 tagy souborů skladeb.

In [2]:
def predict_local(classifier, parameters):
    '''
    This function is used to load and process specified data and predict track genres using selected trained classifier
    Track files can be annotated with ID3 tag
    
    Parameters:
    
    classifier: Full name of the trained classifier file
    parameters: local_archive name and whether to annotate files
    '''
    
    # Initialise variables for storing classifier information
    dataset = ''
    library = ''
    feature_set_name = ''
    clf_name = ''

    info = classifier.split('_')   # Split classifier name 
        
    dataset = info[0]   # Set dataset name
    library = info[1]   # Set feature exraction library name
    
    # Extract other info from classifier name (feature set name and classifier name)
    if len(info) == 5:
        feature_set_name = info[2]
        clf_name = info[3] + '_' + info[4]
    elif len(info) == 6:
        feature_set_name = info[2]
        clf_name = info[3] + '_' + info[4] + '_' + info[5]
    elif len(info) == 7:
        feature_set_name = info[2]
        clf_name = info[3] + '_' + info[4] + '_' + info[5] + '_' + info[6]
    elif len(info) == 9:
        feature_set_name = info[2] + '_' + info[3] + '_' + info[4] + '_' + info[5] + '_' + info[6]
        clf_name = info[7] + '_' + info[8]
    elif len(info) == 10:
        feature_set_name = info[2] + '_' + info[3] + '_' + info[4] + '_' + info[5] + '_' + info[6]
        clf_name = info[7] + '_' + info[8] + '_' + info[9]
    elif len(info) == 11:
        feature_set_name = info[2] + '_' + info[3] + '_' + info[4] + '_' + info[5] + '_' + info[6]
        clf_name = info[7] + '_' + info[8] + '_' + info[9] + '_' + info[10]
    else:
        print('Cannot process classifier name.', file=sys.stderr)
    
    # Load specified extracted features
    try:                
        features = pd.read_csv('../metadata/features/features_{}_{}.csv'.format(parameters['local_archive_name'], library), index_col=0, header=[0, 1, 2])        
    except Exception as e:
        print('Failed to read file: "../metadata/features/features_{}_{}.csv"!'.format(parameters['local_archive_name'], library), file=sys.stderr)
        print('Error: {}'.format(repr(e)), file=sys.stderr)
        return -1
    
    ###################################################################################################################################################

    # Feature beats_position has different length which may cause problems with trained classifiers
    # Comment this out if using classifiers trained on features without beats_position feature
    
    # Load file with correct columns on which classifier was trained
    with open('../metadata/misc/columns_fix.json') as f:
        cols = json.load(f)

    columns = pd.MultiIndex.from_tuples(cols[dataset], names=['feature', 'statistic', 'number'])    
    features = features.reindex(columns, fill_value=0, axis="columns")
    
    ###################################################################################################################################################

    # Perform One-hot encoding on categorical features
    for column in features.select_dtypes(include='object'):   # For each categorical column
        dummy_columns = pd.get_dummies(features[column])   # Encode the column values   
        features = features.drop(columns=column)   # Drop the column from the dataframe

        # Reindex columns to fixed length with all possible instances to avoid feature mismatch with trained classifiers
        if (column[0] in ['chords_key', 'key_edma', 'key_krumhansl', 'key_temperley']) and (column[1] in ['none', 'key']):
            dummy_columns = dummy_columns.reindex(['Ab', 'B', 'Bb', 'C', 'C#', 'D', 'E', 'Eb', 'F', 'F#', 'G'], axis=1, fill_value=0)
        elif (column[0] in ['chords_scale', 'key_edma', 'key_krumhansl', 'key_temperley']) and (column[1] in ['none', 'scale']):
            dummy_columns = dummy_columns.reindex(['minor'], axis=1, fill_value=0)

        # Create correct multiindex for the encoded columns and append them to the features dataframe
        dummy_columns.columns = pd.MultiIndex.from_product([[column[0]], [column[1]], ['{}'.format(c) for c in dummy_columns.columns]], names=features.columns.names)
        features = pd.concat([features, dummy_columns], axis=1).sort_index(axis=1)
        
    # Load track-genre list
    try:                
        genres = pd.read_csv("../metadata/test_data/{}_genres.csv".format(dataset), index_col=0, header=0)        
    except Exception as e:
        print('Failed to read file: "../metadata/test_data/{}_genres.csv"'.format(dataset), file=sys.stderr)
        print('Error: {}'.format(repr(e)), file=sys.stderr)
        return -1
    
    # Fit the encoder to be later able to transform predicted genre values back to actual genres
    encoder = LabelEncoder().fit(np.ravel(genres))
    
    # If optimised feature set is selected, load it, else use all features
    if feature_set_name != 'all':
        
        # Load optimised feature sets if available
        try:
            with open('../metadata/misc/optimised_feature_sets.json') as f:
                optimised_feature_sets = json.load(f)   
        except Exception as e:
            print('Failed to read file: "../metadata/misc/optimised_feature_sets.json"!', file=sys.stderr)
            print('Error: {}'.format(repr(e)), file=sys.stderr)
            return -1
        
        # Remove info about optimisation from classifier name to get correct feature sets key
        if '_default' in clf_name:
            tmp_clf_name = clf_name.replace('_default', '')
        else:
            tmp_clf_name = clf_name[:-13]
        
        feature_set = optimised_feature_sets[dataset][library][tmp_clf_name][feature_set_name]
    else:
        feature_set = features.columns.levels[0]

    scaler = StandardScaler()   # Init the scaler
    
    X = features[feature_set].values   # Extract feature set values to ndarray
    X = scaler.fit_transform(X)   # Scale the features
    
    # Load selected trained classifier
    try:
        clf = joblib.load('../metadata/trained_classifiers/{}_{}_{}_{}.joblib.dat'.format(dataset, library, feature_set_name, clf_name))
    except Exception as e:
        print('Failed to read file: "../metadata/trained_classifiers/{}_{}_{}_{}.joblib.dat"'.format(dataset, library, feature_set_name, clf_name), file=sys.stderr)
        print('Error: {}'.format(repr(e)), file=sys.stderr)
        return -1
    
    print('Predicting with classifier {} trained on {} dataset:\n'.format(clf_name, dataset))
    
    predictions = encoder.inverse_transform(clf.predict(X))   # Transform genre values back to genres
    EasyID3.RegisterTextKey('comment', 'COMM')   # Used to set track file ID3 comment
    
    for track, predicted_genre in dict(zip(list(features.index), predictions)).items():   # For each of the predicted tracks
        print('Track: {: <60} Predicted genre: {}'.format(track, predicted_genre))   # Print the track name/path and its predicted genre
        
        # If annotate tracks is selected, modify tracks ID3 tag
        if parameters['annotate_tracks']:
            audio = EasyID3(track)
            audio['comment'] = 'Genre: {}'.format(predicted_genre)   # Set ID3 comment to be able to see the genre in Nautilus file explorer
            audio['genre'] = predicted_genre   # Set the ID3 genre entry
            audio.save()   # Save the tag
        
    print('\n') 

Tato buňka spustí predikci žánrů u vybraných skladeb pomocí vybraného natrénovaného klasifikačního algoritmu. <br>
ID3 tagy u souborů se skladbami je možné změnit na zákldě predikovaného žánru. <br>
Které klasifikátory mají být využity je možné nastavit v proměnné <code>classifiers</code>, název klasifikátoru musí odpovídat názu souboru, ve kterém je natrénovaný klasifikátor uložen. <br>

Popis parametrů: <br>

<ul>
    <li><code>local_archive_name</code> Název archivu se skladbami, pro které má být predikce provedena. (musí odpovídat názvu použitém při extrakci atributů)</li>
    <li><code>annotate_tracks</code> Hodnota True značí, že ID3 tag u skladeb bude modifikován, aby obsahoval informaci o predikovaném žánru.</li>
</ul>

In [3]:
if __name__ == "__main__":
    
    # Classifiers to be used for predictions (must be a full name of the saved classifier file)
    classifiers = [
        'FMA_essentia_all_XGBClassifier_optimised_VS',
#         'FMA_essentia_opt_feature_set_FS_VS_MLPClassifier_optimised_VS'
    ]
    
    parameters = {
        'local_archive_name': 'local_archive',   # Name of the archive (must match selected name in feature extraction)
        'annotate_tracks': True,   # Annotate track files with genre tags
    }
    
    # For each of the selected classifiers perform predictions
    for classifier in classifiers:
        predict_local(classifier, parameters)

Predicting with classifier XGBClassifier_optimised_VS trained on FMA dataset:

Track: ../data/local_archive/Electronic_new.mp3                     Predicted genre: Experimental
Track: ../data/local_archive/Experimental_new.mp3                   Predicted genre: Experimental
Track: ../data/local_archive/Folk_new.mp3                           Predicted genre: Instrumental
Track: ../data/local_archive/Hiphop_new.mp3                         Predicted genre: Hip-Hop
Track: ../data/local_archive/Instrumental_new.mp3                   Predicted genre: Electronic
Track: ../data/local_archive/International_new.mp3                  Predicted genre: International
Track: ../data/local_archive/Pop_new.mp3                            Predicted genre: International
Track: ../data/local_archive/Rock_new.mp3                           Predicted genre: Rock


